https://github.com/facebookresearch/faiss/blob/main/tutorial/python/1-Flat.py

In [22]:
import numpy as np

In [23]:
d = 64                           # dimension
nb = 100000                      # database size
nq = 10000                       # nb of queries

In [24]:
np.random.seed(1234)             # make reproducible

In [25]:
# for database
xb = np.random.random((nb, d)).astype('float32')
xb.shape

(100000, 64)

In [26]:
# 모든 데이터베이스 0차원에 "np.arange(nb) / 1000" 더하기
xb[:, 0] += np.arange(nb) / 1000.
xb[:, 0]

array([1.91519454e-01, 3.83317441e-01, 7.53425434e-02, ...,
       1.00811470e+02, 1.00669464e+02, 1.00855637e+02], dtype=float32)

In [27]:
# 쿼리 데이터베이스 생성
xq = np.random.random((nq, d)).astype('float32')
xq.shape

(10000, 64)

In [28]:
xq[:, 0] += np.arange(nq) / 1000.
xq[:, 0]

array([ 0.81432974,  0.5844774 ,  0.75040764, ..., 10.96773   ,
       10.831193  , 10.078484  ], dtype=float32)

In [29]:
import faiss

#### What is index and IndexFlatL2?

In [30]:
# Faiss에서 index는 고차원 벡터들을 효과적으로 검색하고 관리하기 위한 핵심 데이터 구조입니다
# The L2 distance, also known as the Euclidean distance,
# 벡터들에 대해서 L2 거리 검색을 수행하는 간단한 버전인 IndexFlatL2를 사용한다

# 대부분의 인덱스는 벡터의 분포를 분석하기 위한 훈련 단계를 포함한다
# 그런데 indexFlatL2는 훈련 단계가 없다

# 인덱스가 생성되고 훈련되면, 주요 두 가지 함수를 사용할 수 있다
# - add: 데이터베이스에 벡터 추가
# - search: 데이터베이스에서 가장 가까운 벡터 검색


In [31]:
index = faiss.IndexFlatL2(d) # 고정된 차원으로 선언
# 훈련 단계가 필요없어서, 훈련이 완료되어서 is_trained가 True가 된다
print(index.is_trained)

True


#### Building an index and adding the vectors to it

In [32]:
# Add vectors to the index
index.add(xb)
print(index.ntotal) # d가 아닌 nb가 나옴

100000


In [46]:
index.d

64

In [48]:
# 저장된 벡터의 수를 가져온다
num_vectors = index.ntotal

# reconstruct 함수를 이용해서 출력할 수 있다
for i in range(num_vectors):
    vec = index.reconstruct(i)
    print(vec)
    # 샘플 출력
    break

[0.19151945 0.62210876 0.43772775 0.7853586  0.77997583 0.2725926
 0.27646425 0.8018722  0.95813936 0.87593263 0.35781726 0.5009951
 0.6834629  0.71270204 0.37025076 0.5611962  0.50308317 0.01376845
 0.7728266  0.8826412  0.364886   0.6153962  0.07538124 0.368824
 0.9331401  0.65137815 0.39720258 0.78873014 0.31683612 0.56809866
 0.8691274  0.4361734  0.8021476  0.14376682 0.70426095 0.7045813
 0.21879211 0.92486763 0.44214076 0.90931594 0.05980922 0.18428709
 0.04735528 0.6748809  0.59462476 0.5333102  0.04332406 0.5614331
 0.32966843 0.5029668  0.11189432 0.6071937  0.5659447  0.00676406
 0.6174417  0.9121229  0.7905241  0.99208146 0.95880175 0.7919641
 0.28525096 0.62491673 0.4780938  0.19567518]


#### Index Searching

In [ ]:
# L2 거리 기반으로 KNN 검색을 수행한다

In [51]:
# k개의 최근접 이웃을 검색한다
k = 4

D, I = index.search(xb[:5], k) # sanity check

# 쿼리 벡터와 데이터베이스 내 해당 쿼리의 k-최근접 이웃 간의 거리를 포함하고 있다
print(D)

[[0.        7.1751738 7.20763   7.2511625]
 [0.        6.3235645 6.684581  6.799946 ]
 [0.        5.7964087 6.391736  7.2815123]
 [0.        7.2779055 7.5279875 7.662846 ]
 [0.        6.7638035 7.2951202 7.3688145]]


In [53]:
#  각 쿼리 벡터에 대한 k-최근접 이웃의 인덱스(또는 ID)를 포함하고 있습니다.
print(I)

[[  0 393 363  78]
 [  1 555 277 364]
 [  2 304 101  13]
 [  3 173  18 182]
 [  4 288 370 531]]


#### Xq를 이용한 Search

In [56]:
D, I = index.search(xq, k) 
print(D[:5])    
print(I[:5])                   

[[6.815506  6.8894653 7.3956795 7.4290257]
 [6.6041145 6.679699  6.7209625 6.828678 ]
 [6.4703865 6.8578644 7.0043755 7.036564 ]
 [5.573681  6.4075394 7.1395187 7.3555946]
 [5.409401  6.232216  6.4173393 6.5743675]]
[[ 381  207  210  477]
 [ 526  911  142   72]
 [ 838  527 1290  425]
 [ 196  184  164  359]
 [ 526  377  120  425]]
